In [1]:
# Activate the environment using the path to the Macro repository
macro_repo_path = dirname(@__DIR__)
using Pkg; Pkg.activate(macro_repo_path)

  Activating project at `~/Code/Macro`


In [2]:
using Macro
using HiGHS

Single zone electricity system with utility scale solar PV, land-based wind generation, natural gas power generation, and electricity storage. The model includes a carbon price of 250 $/ton.

In [3]:
inputs_path = "one_zone";

In [4]:
system = Macro.load_system(inputs_path);

┌ Info: Loading JSON data from /Users/fpecci/Code/Macro/tutorials/one_zone/system_data.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from one_zone/system_data.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from one_zone/system/nodes.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading columns [:natgas_SE] from CSV data from one_zone/system/fuel_prices.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:8
┌ Info: Loading CSV data from one_zone/system/fuel_prices.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading columns [:Demand_MW_z1] from CSV data from one_zone/system/demand.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:8
┌ Info: Loading CSV data from one_zone/system/demand.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:14
┌

system.settings.Scaling = true


┌ Info: Scaling system data to GWh | ktons | M$
└ @ Macro /Users/fpecci/Code/Macro/src/model/scaling.jl:19


In [5]:
model = Macro.generate_model(system)

┌ Info: Starting model generation
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:15
┌ Info: Defining available capacity
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:18
┌ Info: Generating planning model
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:21
┌ Info: Generating operational model
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:24
┌ Info: Model generation complete, it took 4.831251859664917 seconds
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:29


A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: JuMP.AffExpr
├ num_variables: 105133
├ num_constraints: 297861
│ ├ JuMP.AffExpr in MOI.EqualTo{Float64}: 43801
│ ├ JuMP.AffExpr in MOI.GreaterThan{Float64}: 26281
│ ├ JuMP.AffExpr in MOI.LessThan{Float64}: 122647
│ ├ JuMP.VariableRef in MOI.EqualTo{Float64}: 6
│ └ JuMP.VariableRef in MOI.GreaterThan{Float64}: 105126
└ Names registered in the model
  └ :eFixedCost, :eVariableCost, :vREF

In [6]:
Macro.set_optimizer(model, HiGHS.Optimizer);

In [7]:
Macro.optimize!(model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-09, 2e+02]
  Cost   [1e-04, 1e+01]
  Bound  [1e+00, 1e+00]
  RHS    [3e+01, 2e+02]
Presolving model
162464 rows, 92387 cols, 548710 nonzeros  0s
148836 rows, 78759 cols, 526038 nonzeros  0s
Presolve : Reductions: rows 148836(-43893); columns 78759(-26374); elements 526038(-158062)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
      21055     8.7484977714e+04 Pr: 14543(450843); Du: 0(1.07832e-05) 5s
      36262     9.5004939521e+04 Pr: 0(0); Du: 0(1.48822e-13) 7s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 36262
Objective value     :  9.5004939521e+04
Relative P-D gap    :  4.7023217926e-14
HiGHS run time      :          7.27


In [8]:
capacity_results = Macro.get_optimal_asset_capacity(system)

Row,asset,type,capacity,additions,retirements
,Symbol,Symbol,Float64,Float64,Float64
1,battery_SE,Battery,1.75075e5,1.75075e5,0.0
2,SE_natural_gas_fired_combined_cycle_1,ThermalPower{NaturalGas},26218.7,0.0,0.0
3,SE_utilitypv_class1_moderate_70_0_2_1,VRE,5.37868e5,5.37868e5,0.0
4,SE_landbasedwind_class4_moderate_70_1,VRE,2.25881e5,2.25881e5,0.0
